# Basic Example: Prompt+Model+OutputParser

- Author: [ChangJun Lee](https://www.linkedin.com/in/cjleeno1/)
- Design: []()
- Peer Review: [Erika](https://github.com/ErikaPark)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)


## Overview

The most fundamental and commonly used case involves linking a prompt template with a model. To illustrate how this works, let us create a chain that asks for the capital cities of various countries.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Utilizing Prompt Templates](#utilizing-prompt-templates)
- [Chain Creation](#chain-creation)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [6]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [7]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [8]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [9]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv(override=True)

True

In [10]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

True

## Utilizing Prompt Templates

`PromptTemplate`

- A prompt template is used to create a complete prompt string by incorporating the user's input variables.
- Usage
  - `template`: A template string is a predefined format where curly braces '{}' are used to represent variables.

  - `input_variables`: The names of the variables to be inserted within the curly braces are defined as a list.

`input_variables`

- `input_variables` is a list that defines the names of the variables used in the `PromptTemplate`.

In [11]:
from langchain_core.prompts import PromptTemplate

The `from_template()` method is used to create a `PromptTemplate` object.

In [12]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [13]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [14]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [15]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

## Chain Creation

### LCEL (LangChain Expression Language)

Here, we use LCEL to combine various components into a single chain.

![lcel.png](./assets/lcel.png)

```
chain = prompt | model | output_parser
```

The `|` symbol works similarly to the [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), linking different components and passing the output of one component as the input to the next.

In this chain, user input is passed to the prompt template, and the output from the prompt template is then forwarded to the model. By examining each component individually, you can understand what happens at each step.

In [16]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model

### Calling `invoke()`

- Input values are provided in the form of a Python dictionary (key-value pairs).  
- When calling the `invoke()` function, these input values are passed as arguments.

In [17]:
# Set the topic in the `input` dictionary to 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [18]:
# Connect the `prompt` object and the `model` object using the pipe (`|`) operator.  
# Use the `invoke` method to pass the `input`.  
# This will return the message generated by the AI model.
chain.invoke(input)

AIMessage(content='Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can do.\n\n2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want to teach an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. Over time, it learns to identify the features that make a cat a cat.\n\n3. **Feedback Loop**: AI models improve through feedback. When they make predictions or decisions, they can receive feedback on whether they were right or wrong. This feedback helps them adjust and learn from their mistakes, simi

Below is an example of outputting a streaming response:

In [19]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more relevant and high-quality data you provide, the better the model can learn and make predictions.

2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want a model to recognize cats in pictures, you show it many pictures of cats and non-cats. The model learns to identify patterns that distinguish cats from other objects.

3. **Feedback Loop**: Many AI models use a feedback mechanism. After making a prediction, they compare it to the correct answer. If they get it wrong, they adjust their internal settings to improve future predictions. This is similar to how we learn from our mistakes.

4. **Generalization**: A good AI model doesn’t just memorize the examples it sees; it learns t

### Output Parser

An **Output Parser** is a tool designed to transform or process the responses from an AI model into a specific format. Since the model's output is typically provided as free-form text, an **Output Parser** is essential to convert it into a structured format or extract the required data.


In [20]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser() # Directly returns the model's response as a string without modification.

An output parser is added to the chain.

In [21]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser

In [22]:
# Use the invoke method of the chain object to pass the input
chain.invoke(input)

'Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:\n\n1. **Data Input**: AI models learn from data. The more relevant and high-quality data they receive, the better they can learn. Think of it like a student studying from a good textbook.\n\n2. **Patterns Recognition**: AI looks for patterns in the data. Just like humans recognize faces or voices, AI identifies trends and relationships in the information it processes.\n\n3. **Feedback Loop**: AI models often use feedback to improve. When they make predictions or decisions, they can receive feedback on whether they were right or wrong. This is similar to a teacher correcting a student’s mistakes.\n\n4. **Generalization**: After learning from specific examples, AI models try to apply what they’ve learned to new, unseen data. This is like a stu

In [23]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more relevant and high-quality data you provide, the better the model can learn. Think of it like teaching a child; the more books and experiences they have, the more they learn.

2. **Patterns and Features**: AI looks for patterns in the data. It identifies important features (characteristics) that help it understand and make predictions. For example, if you're teaching an AI to recognize cats, it will learn features like fur texture, ear shape, and eye color.

3. **Feedback Loop**: Learning involves feedback. When an AI makes a prediction, it gets feedback on whether it was right or wrong. This helps the model adjust and improve over time, similar to how a student learns from their mistakes.

4. **Generalization**: A good AI m

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The `model_name` can also be adjusted for testing.

In [24]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

# Generate the prompt using the PromptTemplate
prompt = PromptTemplate.from_template(template)

# Initialize the ChatOpenAI model.
model = ChatOpenAI(model_name="gpt-4o-mini")

# Initialize the string output parser.
output_parser = StrOutputParser()

In [25]:
# Construct the chain.
chain = prompt | model | output_parser

In [26]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))

- Dialogue in English:
**Waiter:** Good evening! Welcome to Bella Italia. How many people are in your party?  
**You:** Hi! Just one, please.  
**Waiter:** Right this way. Here’s your menu. Can I start you off with something to drink?  
**You:** Yes, I’d like a sparkling water, please.  
**Waiter:** Great choice! Are you ready to order, or do you need a few more minutes?  
**You:** I think I’m ready. I’ll have the spaghetti carbonara, please.  
**Waiter:** Excellent choice! Would you like to add a side salad with that?  
**You:** Yes, that sounds good. I’ll take the house salad.  
**Waiter:** Perfect! I’ll get that order in for you. Is there anything else you need?  
**You:** No, that will be all for now, thank you!  
**Waiter:** You’re welcome! I’ll be back with your drink shortly.  

- Explanation of the Dialogue:  
In this conversation, you are at a restaurant and interact with the waiter. The dialogue begins with the waiter greeting you and asking how many people are in your party.

In [27]:
# Execute the completed Chain to obtain a response  
# Request for Streaming Output
answer = chain.stream({"question": "I want to go to a restaurant and order food."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:

**Waiter:** Good evening! Welcome to Bella’s Bistro. How many are in your party tonight?

**Customer:** Good evening! It’s just me, please. Could I have a table for one?

**Waiter:** Of course! Right this way. Here’s the menu. Can I start you off with something to drink?

**Customer:** Yes, please. I’d like a glass of water and a glass of red wine.

**Waiter:** Great choice! I’ll be right back with your drinks. Have you had a chance to look at the menu?

**Customer:** Yes, I have. I think I’ll have the grilled salmon with asparagus, please.

**Waiter:** Excellent choice! Would you like any appetizers with that?

**Customer:** Hmm, I’ll try the bruschetta, please.

**Waiter:** Perfect! I’ll get that order in for you. Is there anything else I can bring you?

**Customer:** No, that will be all for now, thank you.

**Waiter:** You’re welcome! I’ll be back shortly with your drinks and food.

---

- Explanation of the Dialogue: 

In this dialogue, the customer is at a

In [28]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.  
# Request for Streaming Output
answer = chain.stream({"question": "Ordering Pizza in the US"})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:
**Customer:** Hi there! I’d like to order a pizza, please.  
**Pizza Place:** Sure! What size would you like?  
**Customer:** I’ll have a large, please.  
**Pizza Place:** Great choice! What toppings do you want?  
**Customer:** Can I get pepperoni, mushrooms, and extra cheese?  
**Pizza Place:** Absolutely! Would you like anything else?  
**Customer:** Yes, could I also get a side of garlic bread?  
**Pizza Place:** Of course! That will be $25. Would you like to pick it up or have it delivered?  
**Customer:** I’ll have it delivered, please.  
**Pizza Place:** No problem. Can I have your address?  
**Customer:** Sure! It’s 123 Maple Street.  
**Pizza Place:** Thank you! Your order will be there in about 30 minutes.  
**Customer:** Perfect, thanks so much!  

- Explanation of the Dialogue: 
In this conversation, the customer is placing an order for a pizza at a pizzeria. The interaction starts with the customer greeting the staff and expressing the desire to orde